In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits
import healpy as hp
from astropy import wcs

from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
from multiprocessing import Pool

from scipy import stats

In [2]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [18]:
image_dir = '/global/cfs/cdirs/cosmo/staging/'
surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.fits.gz'

# image_path_list = glob.glob(os.path.join(image_dir, '*ooi*.fits.fz'))
ccd = Table(fitsio.read(surveyccd_path))

In [19]:
# mask = ccd['ccd_cuts']==0
# ccd = ccd[mask]

In [60]:
# Only keep unique exposures
_, idx = np.unique(ccd['expnum'], return_index=True)
ccd = ccd[idx]
print(len(ccd))

36698


In [35]:
ccdnum_list = [1, 2, 3, 4]
ccd_ra = [0.2813, 0.2813, -0.2813, -0.2813]
ccd_dec = [0.263, -0.263, 0.263, -0.263]

binsize = 8
pix_size = 0.454/3600*binsize
vrange = 1.

In [36]:
# mask = np.array(['ksb_160604_055226' in tmp for tmp in ccd['image_filename']])
# print(np.sum(mask))
# ccd_index = np.where(mask)[0][0]
# print(ccd['expnum'][ccd_index], ccd_index)

In [37]:
# mask = np.array(['CP20160603' in tmp for tmp in ccd['image_filename']])
# print(np.sum(mask))
# idx = np.where(mask)[0]

In [61]:
mask = np.array(['CP20160603' in tmp for tmp in ccd['image_filename']])
print(np.sum(mask))
print(np.sum(ccd['ccd_cuts'][mask]==0))
idx = np.where(mask)[0]

170
157


In [27]:
for tmp in ccd['image_filename'][mask]:
    print(tmp)

90prime/CP/V2.3/CP20160603/ksb_160604_032631_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_033607_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_034115_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_034741_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_035338_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_035950_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_040612_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_041123_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_041938_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_042446_ooi_g_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_044113_ooi_r_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_044621_ooi_r_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_045136_ooi_r_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_045503_ooi_r_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_045913_ooi_r_ls9.fits.fz
90prime/CP/V2.3/CP20160603/ksb_160604_050138_ooi_r_ls9.

In [41]:
# idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603-all.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

10312 g 75430031
10313 g 75430034
10314 g 75430035
10315 g 75430036
10316 g 75430037
10317 g 75430038
10318 g 75430039
10319 g 75430040
10320 g 75430042
10321 g 75430043
10322 r 75430048
10323 r 75430049
10324 r 75430050
10325 r 75430051
10326 r 75430052
10327 r 75430053
10328 r 75430054
10329 r 75430055
10330 r 75430056
10331 r 75430057
10332 r 75430060
10333 r 75430061
10334 r 75430062
10335 r 75430063
10336 r 75430064
10337 r 75430065
10338 r 75430066
10339 r 75430067
10340 r 75430068
10341 r 75430069
10342 r 75430070
10343 r 75430071
10344 r 75430072
10345 r 75430073
10346 r 75430074
10347 r 75430075
10348 r 75430076
10349 r 75430077
10350 r 75430078
10351 r 75430079
10352 r 75430080
10353 r 75430082
10354 r 75430083
10355 r 75430084
10356 r 75430087
10357 r 75430088
10358 r 75430089
10359 r 75430090
10360 r 75430091
10361 r 75430092
10362 r 75430093
10363 r 75430094
10364 r 75430095
10365 r 75430096
10366 r 75430097
10367 r 75430098
10368 r 75430099
10369 r 75430100
10370 r 754301

In [65]:
expnum_min = 75430073
expnum_max = 75430126

mask = (ccd['expnum']>=expnum_min) & (ccd['expnum']<=expnum_max)
print(np.sum(mask))
expnum_list = np.unique(ccd['expnum'][mask])
print(len(expnum_list))

idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603-all-add_odd_mask.pdf') as pdf:
    
    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

51
51
10345 r 75430073
10346 r 75430074
10347 r 75430075
10348 r 75430076
10349 r 75430077
10350 r 75430078
10351 r 75430079
10352 r 75430080
10353 r 75430082
10354 r 75430083
10355 r 75430084
10356 r 75430087
10357 r 75430088
10358 r 75430089
10359 r 75430090
10360 r 75430091
10361 r 75430092
10362 r 75430093
10363 r 75430094
10364 r 75430095
10365 r 75430096
10366 r 75430097
10367 r 75430098
10368 r 75430099
10369 r 75430100
10370 r 75430101
10371 r 75430102
10372 r 75430103
10373 r 75430104
10374 r 75430105
10375 r 75430106
10376 r 75430107
10377 r 75430108
10378 r 75430109
10379 r 75430110
10380 r 75430111
10381 r 75430112
10382 r 75430113
10383 r 75430114
10384 r 75430115
10385 r 75430116
10386 r 75430117
10387 r 75430118
10388 r 75430119
10389 r 75430120
10390 r 75430121
10391 r 75430122
10392 r 75430123
10393 r 75430124
10394 r 75430125
10395 r 75430126


In [62]:
ccd_ok = Table(fitsio.read(surveyccd_path))
mask = ccd_ok['ccd_cuts']==0
ccd_ok = ccd_ok[mask]
print(len(ccd_ok))

132446


In [64]:
expnum_min = 75430073
expnum_max = 75430126

mask = (ccd['expnum']>=expnum_min) & (ccd['expnum']<=expnum_max)
print(np.sum(mask))
expnum_list = np.unique(ccd['expnum'][mask])
print(len(expnum_list))

idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603-all_ccd_cuts.pdf') as pdf:

    for iii, ccd_index in enumerate(idx):
        
        print(iii, '/', len(idx))

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)
        
        mask = (ccd_ok['expnum']==expnum)
        if np.sum(mask)==0:
            print('no good CCD in {}'.format(expnum))
            continue

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):
            
            mask = (ccd_ok['expnum']==expnum) & (ccd_ok['ccdname']=='CCD'+str(ccdnum))
            if np.sum(mask)==0:
                print('CCD'+str(ccdnum)+' not available in {}'.format(expnum))
                continue

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

51
51
 expnum 
--------
75430073
75430074
75430075
75430076
75430077
75430078
75430079
75430080
75430082
75430083
     ...
75430116
75430117
75430118
75430119
75430120
75430121
75430122
75430123
75430124
75430125
75430126
Length = 51 rows
0 / 51
10345 r 75430073
1 / 51
10346 r 75430074
2 / 51
10347 r 75430075
3 / 51
10348 r 75430076
4 / 51
10349 r 75430077
5 / 51
10350 r 75430078
6 / 51
10351 r 75430079
7 / 51
10352 r 75430080
8 / 51
10353 r 75430082
9 / 51
10354 r 75430083
10 / 51
10355 r 75430084
11 / 51
10356 r 75430087
12 / 51
10357 r 75430088
13 / 51
10358 r 75430089
14 / 51
10359 r 75430090
15 / 51
10360 r 75430091
16 / 51
10361 r 75430092
17 / 51
10362 r 75430093
18 / 51
10363 r 75430094
19 / 51
10364 r 75430095
20 / 51
10365 r 75430096
21 / 51
10366 r 75430097
22 / 51
10367 r 75430098
23 / 51
10368 r 75430099
24 / 51
10369 r 75430100
25 / 51
10370 r 75430101
26 / 51
10371 r 75430102
27 / 51
10372 r 75430103
28 / 51
10373 r 75430104
29 / 51
10374 r 75430105
30 / 51
10375 r 75430

In [10]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603-all.pdf') as pdf:

    for ccd_index in idx[::5]:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

9057 g 75430038
9062 r 75430050
9067 r 75430055
9072 r 75430062
9077 r 75430067
9082 r 75430072
9087 r 75430077
9092 r 75430083
9097 r 75430090
9102 r 75430095
9107 r 75430100
9112 r 75430105
9117 r 75430110
9122 r 75430115
9127 r 75430120
9132 r 75430125
9137 r 75430133
9142 r 75430138
9147 r 75430145
9152 r 75430150
9157 r 75430155
9162 r 75430160
9167 r 75430166
9172 r 75430171
9177 r 75430176
9182 r 75430181
9187 r 75430186
9192 r 75430192
9197 r 75430197
9202 r 75430202
9207 r 75430207
9212 r 75430212


In [34]:
# idx = np.where(ccd['expnum']==79240048)[0][0]
idx1 = np.concatenate([np.arange(idx.min()-6, idx.min()), np.arange(idx.max()+1, idx.max()+6)])
print(idx1)

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603_neighbors.pdf') as pdf:

    for ccd_index in idx1:

        # from matplotlib.backends.backend_pdf import PdfPages
        # with PdfPages('plots/90prime_images-ccd4.pdf') as pdf:
        # for ccd_index in range(idx-20, idx+20):

        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)
        fn = ccd['image_filename'][ccd_index]

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

[9051 9052 9053 9054 9055 9056 9214 9215 9216 9217 9218]
9051 g 75420170
9052 g 75420171
9053 g 75420172
9054 g 75420173
9055 g 75420174
9056 g 75420175
9214 r 75440042
9215 r 75440043
9216 r 75440044
9217 r 75440045
9218 r 75440046
